# Import Packages and Files

In [8]:
import torch.nn as nn
import dataloader as dl
import saveloader as sl
import rotation as rot
import RotNet as RN
import trainer as tr

# Load Dataset

In [3]:
trainset, testset, classes = dl.load_cifar("./datasets")

Files already downloaded and verified
Files already downloaded and verified


In [4]:
trainloader, validloader, testloader = dl.make_dataloaders(trainset, testset, 8000, 128)

# Train RotNet for Rotation Task

In [7]:
# initialize network
net = RN.RotNet(num_classes=4, num_conv_block=4, add_avg_pool=False)

rot_classes = ['original', '90 rotation', '180 rotation', '270 rotation']
criterion = nn.CrossEntropyLoss()

In [ ]:
rot_loss_log, rot_accuracy_log, rot_max_accuracy, rot_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], [60, 120, 160, 200], 0.9, 5e-4, net, trainloader, validloader, criterion, rot=['90', '180', '270'])